In [1]:
# 


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 23:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 17:28:51,744 INFO: Initializing external client
2025-03-05 17:28:51,751 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 17:28:52,635 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [6]:
df= query.read()

2025-03-05 17:29:49,694 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 301, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\six.py", line 724, in reraise
    raise value
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 251, in call
    attempt = Attempt(fn(

2025-03-05 17:29:55,122 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 301, i

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
df

In [ ]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

In [ ]:
now = datetime.now(timezone.utc)

In [ ]:
predictions

In [ ]:
df.info()

In [ ]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [ ]:
dt = current_date.ceil('h')


In [ ]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

In [ ]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

In [ ]:
results

In [ ]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [ ]:
predictions = fetch_next_hour_predictions()

In [ ]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



In [ ]:
df = fg.read()

In [ ]:
df["pickup_hour"].max()

In [ ]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

In [ ]:
df[df["pickup_hour"] == next_hour]